In [1]:
!pip install tensorflow

In [2]:
import tensorflow as tf

# Verificar si la aceleración de GPU está habilitada
if tf.config.list_physical_devices("GPU"):
    print("Tienes acceso a la aceleración de GPU")
else:
    print("No tienes acceso a la aceleración de GPU")

Tienes acceso a la aceleración de GPU


In [3]:
!pip install pycuda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 26.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 9.7 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2022.2.2-cp310-cp310-linux_x86_64.whl size=661265 sha256=5288ee910da07f2944ea64300b57a816f0261c5681f4701aa4dc86cb333ba3e3
  Stored in directory: /root/.cache/pip/wheels/1d/7b/06/82a395a243fce00035dea9914d92bbef0013401497d849f8bc
Successfully built pycuda


In [4]:
!pip install gputil

  Preparing metadata (setup.py) ... done
  Created wheel for gputil: filename=GPUtil-1.4.0-py3-none-any.whl size=7392 sha256=d6b1c530a31cc647c3a23a6e9ed67e0167b65b7d284e697c6fa286edf5d0c451
  Stored in directory: /root/.cache/pip/wheels/a9/8a/bd/81082387151853ab8b6b3ef33426e98f5cbfebc3c397a9d4d0
Successfully built gputil


In [5]:
!pip install py3nvml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 902.4 kB/s eta 0:00:00


In [6]:
!nvidia-smi

Sun Sep  3 02:51:53 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8     9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [21]:
import pycuda.autoinit  # Inicializa PyCUDA para trabajar con la GPU
import pycuda.driver as cuda  # Proporciona acceso a la GPU
import pycuda.compiler
import numpy as np  # Librería para trabajar con arreglos multidimensionales
import GPUtil  # Librería para obtener información de la GPU

In [22]:
# Función para realizar el ordenamiento "Select Sort" en la CPU
def select_sort(arr):
    n = len(arr)
    for i in range(n):
        min_idx = i
        for j in range(i+1, n):
            if arr[j] < arr[min_idx]:
                min_idx = j
        arr[i], arr[min_idx] = arr[min_idx], arr[i]


In [23]:
# Define el kernel CUDA para encontrar el número mayor
kernel_code = """
__global__ void find_max(const int *arr, int *result, int n) {
    // Trabajo con hilos y bloques en paralelo
    int tid = threadIdx.x + blockIdx.x * blockDim.x;
    int local_max = arr[tid];

    for (int i = tid + blockDim.x; i < n; i += blockDim.x * gridDim.x) {
        if (arr[i] > local_max) {
            local_max = arr[i];
        }
    }

    // Guarda el resultado en el bloque
    result[blockIdx.x] = local_max;
}
"""


In [24]:
# Tamaño del arreglo
n = 956

# Genera un arreglo de números aleatorios en la CPU
arr_cpu = np.random.randint(0, 100000, size=n, dtype=np.int32)

In [25]:
# Ordena el arreglo utilizando Selection Sort en la CPU
select_sort(arr_cpu)

# Copia el arreglo ordenado desde la CPU a la GPU
arr_gpu = cuda.mem_alloc(arr_cpu.nbytes)
cuda.memcpy_htod(arr_gpu, arr_cpu)

In [26]:
# Configura los hilos y bloques en la GPU
block_size = 90
grid_size = (n + block_size - 1) // block_size

# Crea un arreglo para almacenar los resultados por bloque en la GPU
block_results = np.zeros(grid_size, dtype=np.int32)

In [27]:
# Crea el kernel (función CUDA) que se ejecutará en la GPU
module = SourceModule(kernel_code)
find_max_kernel = module.get_function("find_max")

NameError: ignored

In [ ]:
# Realiza múltiples llamadas al kernel para aumentar la carga de trabajo
for _ in range(10000):  # Realiza 1000 veces la búsqueda del número máximo
    find_max_kernel(arr_gpu, cuda.Out(block_results), np.int32(n), block=(block_size, 1, 1), grid=(grid_size, 1, 1))

# Obtiene información de la GPU después de la ejecución
after_info = GPUtil.getGPUs()[0]

In [ ]:
# Imprime la información de uso de la GPU después de la ejecución
print("\nInformación de la GPU después de la ejecución:")
print(f"  Uso de GPU: {after_info.load * 100}% (estimado)")
print(f"  Memoria usada: {after_info.memoryUsed} MB")
print(f"  Memoria libre: {after_info.memoryFree} MB")
print(f"  Temperatura: {after_info.temperature}°C")

In [ ]:

# Recupera el resultado de la GPU (arreglo con el máximo local de cada bloque)
gpu_max_results = np.copy(block_results)

# Encuentra el máximo global a partir de los máximos locales
global_max = np.max(gpu_max_results)

# Imprime el número mayor encontrado
print(f"\nNúmero mayor en la matriz: {global_max}")

NameError: name 'SourceModule' is not defined